<a href="https://colab.research.google.com/github/baldpanda/advent-of-haystack-2023/blob/main/day_9/advent_of_haystack_day_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Haystack - Day 9
_Make a copy of this Colab to start_

In this challenge, your task is to transcribe an audio file and convert the style to sound more like Santa! You will be provided with:
1. **Some different LLM options:** You can chose to use the `HuggingFaceLocalGenerator` (which we've provided the code for, using Zephyr 7B), `HuggingFaceTGIGenerator` (which we've provided the code for using the new Mixtral-8x7B-Instruct-v0.1 model), or, you can also chose to use a GPT model with the `GPTGenerator`.
2. **An audio file:** There is a code cell that you should run which will fetch an audio file and store it in the "Files" section of this Colab. Enjoy 🎸
3. **Some imports:** Pay attention to what we've imported in the code cell where you will complete the challenge. This might give you some hints!
4. **A pipeline.run() cell:** At the end of the colab, you will see a `pipeline.run()` cell. This will help you understand how we expect the pipeline to be created.

**💚 Some Hints**
- The [`LocalWhisperTranscriber`](https://docs.haystack.deepset.ai/v2.0/docs/localwhispertranscriber): This component expects `audio_files` as input, and returns a list of `documents`. It will be very useful for this challenge!

#Installation
**Note:** There is a known issue with colab due to a version conflict error related to `llmx` which comes with Colab. You might get an `llmx` error. You can safely ignore this, or run `pip uninstall -y llmx`

In [1]:
!pip install haystack-ai
!pip install transformers[torch] accelerate bitsandbytes
!pip install openai-whisper boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━

### Enabling Telemetry

Knowing you’re running this challenge helps us know whether Advent of Haystack is helping people learn about Haystack 2.0-Beta. But you can always opt out by commenting the following line.

In [2]:
from haystack.telemetry import tutorial_running

tutorial_running("challenge_9")

## 1) Select the LLM you would like to use 👇


Here, we've provided you with 2 options. The `HuggingFaceTGIGenerator` with the Mixtral 8x7B model is a runnable code cell. If you would like to switch to another model, make sure to copy over the code from the markdown cell and re-run it.

For the `HuggingFaceTGIGenerator` we are using Hugging Face [Text Generation interface](https://github.com/huggingface/text-generation-inference) (TGI). This required an API key 👇 [Getting your API key](https://huggingface.co/docs/api-inference/quicktour#get-your-api-token)

In [3]:
from haystack.components.generators import HuggingFaceTGIGenerator
from getpass import getpass

hf_token = getpass("Enter Hugging Face API key:")

llm = HuggingFaceTGIGenerator("mistralai/Mixtral-8x7B-Instruct-v0.1", token=hf_token)
llm.warm_up()

Enter Hugging Face API key:··········


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Use the code below to switch to another model. Or, check out the [`GPTGenerator`](https://docs.haystack.deepset.ai/v2.0/docs/gptgenerator) docs for OpenAI model options.

```python
import torch
from haystack.components.generators import HuggingFaceLocalGenerator
llm = HuggingFaceLocalGenerator("HuggingFaceH4/zephyr-7b-beta",
                                 huggingface_pipeline_kwargs={"device_map":"auto",
                                                   "model_kwargs":{"load_in_4bit":True,
                                                                   "bnb_4bit_use_double_quant":True,
                                                                   "bnb_4bit_quant_type":"nf4",
                                                                   "bnb_4bit_compute_dtype":torch.bfloat16}})
llm.warm_up()
```

## 2) Transcribe the audio file and make it "Santa style" 🧑‍🎄

### Step 1: First, run the cell below to fetch the audio file. This will store it in the "Files" tab on this Colab

In [4]:
# @title Default title text
import boto3
from botocore import UNSIGNED
from botocore.config import Config
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

s3.download_file('core-engineering', 'public/advent/audio.mp3', '/content/audio.mp3')

### Step 2: Create the pipeline for transcribing and styling

In [5]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.audio import LocalWhisperTranscriber

###### Create a pipeline
prompt_template = """
{% for doc in documents %}
  Please make the following document more Christmassy: {{ doc.content }} \n
{% endfor %};
"""

whisper = LocalWhisperTranscriber(model_name_or_path="small")
prompt_builder = PromptBuilder(template=prompt_template)

pipeline = Pipeline()
pipeline.add_component(instance=whisper, name="transcriber")
pipeline.add_component(instance=prompt_builder, name="prompt_builder")
pipeline.add_component(instance=llm, name="llm")

pipeline.connect("transcriber.documents", "prompt_builder.documents")
pipeline.connect("prompt_builder", "llm")



### Step 3: Run the pipeline

Here you'll see how we expect the pipeline tu be run. Be sure to change the names of the components in the cell below if you have named them differently in your pipeline above.

In [6]:
result = pipeline.run({"transcriber": {"audio_files": ["/content/audio.mp3"]},
                       "llm":{"generation_kwargs": {"max_new_tokens": 150}}})

print(result["llm"]["replies"][0])

100%|████████████████████████████████████████| 461M/461M [00:03<00:00, 136MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:113: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


)

Here's a more Christmassy version of the document:

'Tis the season of joy and cheer,
And we're no strangers to love this time of year.
You know the rules and so do I,
As we come together to celebrate and be merry.

So let's deck the halls and trim the tree,
And sing carols loud and clear.
For this is the season of giving and receiving,
And we're no strangers to love, my dear.

So let's raise a glass and toast to the season,
And make this Christmas one to remember.
For we're no strangers to love, you see,

